# Workflow Example 1

This is based on a simple workflow, where only one cube is chosen with one dimension to plot over and a restricted number of customization options.

This is not a typical workflow, but it's a good start.

In [ ]:
import ipywidgets
import IPython.display
import iris

import glob
import numpy as np
import iris.quickplot as iplt
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

# 1. Choose and load your cube

Create cube picker consiting of a text acceptor for the file path and a multiple selection pane for the cube selector

In [ ]:
path = ipywidgets.Text(
    description='String:',
    value='/tmp',
)
IPython.display.display(path)

In [ ]:
options = glob.glob('{}/*'.format(path.value)) # Can we do this in a different form, so that we can see the options 
                                               # without the file path?

files = ipywidgets.SelectMultiple(
    description=path.value,
    options=options
)
IPython.display.display(files)

At this point we need to see the cube or cubelist in order to make a decision about what to do next.  For example, if we have one cube then we can move on to the next stage (choosing the plot type) whereas if we have a list of cubes then we have to choose one of those cubes to carry forward and choose the plot type for.  Then, of course, there is the possibility that a user would want to plot several cubes with the same types and settings, so maybe this stage also needs a multiple cube selector.

This would be much easier with a file browser or some equivalent, which would allow the user to see what they are selecting (i.e. whether it is a folder or a file) and then have more control over the cubes they are selecting.  Then later on in the workflow, an option to plot another cube could follow.  This, however, would not allow for multiple plots and plot types with a single associated slider.

Hmmm......

In [ ]:
cubes = files.value

for i in range(len(cubes)):
    print cubes[i]
    if cubes[i][-2:] == 'nc' or cubes[i][-2:] == 'pp': # This is currently not taking grib files into 
                                                           # account coz they are hard to identify.  Also no ff.
        cubelist = iris.load_raw(cubes[i])
    else:
        print 'This is not a cube, please try another selection.'

print cubelist        

Select which cubes you would like to use if you have loaded a cubelist

In [ ]:
to_plot = [cube.standard_name for cube in cubelist]

plots = ipywidgets.SelectMultiple(
description='Choose cube(s)',
options=to_plot
)

IPython.display.display(plots)

I feel that at this point we need a 'continue' button.  In actual fact we probably need several of these at various points in the workflow.  I am coming round to this way of thinking because at nearly every selection point, a dependency is created, so before the next set of options can be displayed, the current selection must be set in order to generate those options.

If this was not the case, I can't see how this is going to work.

You could offer only options which are applicable to every case, which is incredibly limited, or you could offer every single option available in every case, which would chuck out an error in many cases.  Neither of these feel ideal.

# 2. Choose your plot options

Create plot-type picker

In [ ]:
plot_type_dict = {'contour': iplt.contour, 'contourf': iplt.contourf, 'pcolor': iplt.pcolor, 'outline': iplt.outline,
                  'pcolormesh': iplt.pcolormesh, 'plot': iplt.plot, 'points': iplt.points}

plot_types = plot_type_dict.keys()
plot_types.sort()

type = ipywidgets.Dropdown(
    options=plot_types,
    value='contour',
    description='Plot-type:')

IPython.display.display(type)

You will be able to use this later using:
```python
callable = plot_type_dict[type.value]
callable(plots.value)
```

Create axis coordinate pickers for x and y axis

In [ ]:
base_cube = iris.load_cube(files.value, plots.value[0])
# base_cube is the first cube chosen from the list.
# This method will work provided that all the cubes chosen have the same coordinates.

coordinates = [(coord.name()) for coord in base_cube.coords()]
for i in range(len(plots.value)):
    coordinates.append(plots.value[i])

dim_x = ipywidgets.RadioButtons(
    description='Dimension for x:',
    options=coordinates)

IPython.display.display(dim_x)

In [ ]:
if dim_x.value in coordinates:
    coordinates.remove(dim_x.value)

dim_y = ipywidgets.RadioButtons(
    description='Dimension for y:',
    options=coordinates)

IPython.display.display(dim_y)

In [ ]:
# coordinates.remove(dim_y.value)

# slider = ipywidgets.RadioButtons(
#     description='Dimension for slider:',
#     options=coordinates)

# IPython.display.display(slider)


In [ ]:
# index = ipywidgets.IntSlider(
#     value = 0,
#     min=0,
#     max=len(cube.coord(slider.value).points-1),
#     step=1,
#     description='Index of ' + slider.value)

# #IPython.display.display(index)


I had originally made a menu to choose which dimension to slide through, but it has occurred to me that you may want to slide through all dimensions.  This will probably be easier than choosing a dimension and then slicing all other dimensions, as this would probably have to assume an index to slice.  Otherwise, this is going to get even more complex.

In [ ]:
if dim_y.value in coordinates:
    coordinates.remove(dim_y.value)

index = []
for i in range(len(coordinates)):
    coord_index = ipywidgets.IntSlider(
        value=0,
        min=0,
        max=len(base_cube.coord(coordinates[i]).points-1),
        step=1,
        description='Index of ' + coordinates[i])
    index.append(coord_index)
    IPython.display.display(index[i])

# Choose your formatting options

Here we need to take stock of which type of plot has been chosen, and offer an appropriate set of formatting options for that plot type.  This could take a bit of work to be complete and comprehensive.

A big problem that I am having is that some options can be either a boolean or a set value, or a string, or a float, etc., and the list of options is really too long to offer as a dropdown.  It might be possible to add a string entry widget to the options of a drowdown widget, but this is going to make the code very complicated; probably unnecessarily so.  This means that the input options are going to be very restricted at this stage.

In [136]:
# All:
corner_mask = ipywidgets.Dropdown(
    options=['True', 'False', 'legacy'],
    value='True',
    description='corner_mask')
colors = ipywidgets.Dropdown(
    options=['None', 'blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white'],
    value='blue',
    description='colors')
cmap = ipywidgets.Dropdown(
    options=['None', 'viridis', 'inferno', 'plasma', 'magma', 'jet', 'summer', 'autumn'],
    value='None',
    description='cmap')
norm = ipywidgets.Dropdown(
    options=['None', 'Autoscale(A)', 'Autoscale_None(A)', 'scaled()'],
    value='None',
    description='norm')
levels = ipywidgets.Text(
    description='levels',
    value='np.linspace(np.min(cube.data), np.max(cube.data), 10)')
origin = ipywidgets.Dropdown(
    options=['None', 'upper', 'lower', 'image'],
    value='None',
    description='origin')
extend = ipywidgets.Dropdown(
    options=['neither', 'both', 'min', 'max'],
    value='both',
    description='extend')
# Contour-only:
linestyles = ipywidgets.Dropdown(
    options=['None', 'solid', 'dashed', 'dashdot', 'dotted'],
    value='None',
    description='linestyle')
# Plot only:
label = ipywidgets.Text(
    description='label',
    value='Line Plot')
# Plot and Points only:
color = ipywidgets.Dropdown(
    options=['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white'],
    value='blue',
    description='color')
marker = ipywidgets.Dropdown(
    options=['-', '--', '-.', ':', '.', ',', 'o', 'v', '^', '<', '>', '*', 'x'],
    value='-',
    description='marker')
    
formatting = [corner_mask, colors, cmap, norm, levels, origin, extend]
    
if type.value in ['contourf', 'outline', 'pcolor', 'pcolormesh']:
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'contour':
    formatting.append(linestyles)
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'plot':
    formatting.append(label)
    formatting.append(color)
    formatting.append(marker)
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'points':
    formatting.remove(colors)
    formatting.append(color)
    formatting.append(marker)    
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
    

# Make your plot

This will require a button which does all the things that make a plot when you press it.

In [142]:
# Making the plot will take some more work because the divergent options will create a lot of variables here which may 
# or may not be used.  This means that the further we get into dependecies and the more we have in a notebook, the more
# if statements will have to be made in the call to plot.

# This looks like it could get very messy...


callable = plot_type_dict[type.value]

## This should not be necessary as the x-value is removed for the list of y-values, but I will leave it in just in case.
# if dim_x.value == dim_y.value:
#     print 'You are plotting the same coordinate on x and y.  Did you mean to do that?  \
#         You should change one axis coordinate because this plot makes no sense.'
# else:
#     for j in range(len(plots.value)):
#         print plots.value[j]
#         callable(plots.value[j])
#         iplt.show()

plot_opts = {'corner_mask':corner_mask.value, 'colors':colors.value, 'cmap':cmap.value, 
                 'norm':norm.value, 'levels':eval(levels.value), 'origin':origin.value, 'extend':extend.value}
if type.value == 'contour':
    plot_opts['linestyles'] = linestyles.value
elif type.value == 'plot':
    plot_opts['label'] = label.value
    plot_opts['color'] = color.value
    plot_opts['marker'] = marker.value   
elif type.value == 'points':
    del plot_opts['colors']
    plot_opts['color'] = color.value
    plot_opts['marker'] = marker.value    

for j in range(len(plots.value)):
    cube = iris.load_cube(files.value, plots.value[j])
    # Need to find a way of extracting the value of the slice from the slider and plotting it here.
    # I have two problems here.
    # The first is that the plot will need to change WITH the slider. 
    # The second is that the plotting routine only looks for dimension coordinates, but the slider list also contains
    # auxiliary coordinates.
    callable(cube, **plot_opts)
    iplt.show()

surface_temperature / (K)           (time: 54; latitude: 18; longitude: 432)
     Dimension coordinates:
          time                           x             -              -
          latitude                       -             x              -
          longitude                      -             -              x
     Auxiliary coordinates:
          forecast_reference_time        x             -              -
     Scalar coordinates:
          forecast_period: 0 hours
     Attributes:
          Conventions: CF-1.5
          STASH: m01s00i024
     Cell methods:
          mean: month, year


ValueError: Cube must be 2-dimensional. Got 3 dimensions.